In [2]:
#import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.tsaplots import plot_acf
import os
import scipy.optimize as optimize
import scipy.stats as stats
from scipy.stats import laplace_asymmetric

In [3]:
''' Data Generation '''
################################'Generate X and Y'##############################
################################ Xt ~ N(0,1)
################################ Yt = α + βYt-1 + γXt + εt + θεt-1
def generate_data(T, alpha_y, beta_y, gamma, theta, sigma_y, mu, seed=None):
    np.random.seed(seed)
    df = pd.DataFrame(index=range(T), columns=['x', 'y', 'epsilon_y'])
    x = np.random.normal(0, 1, T)
    y1 = np.random.normal(mu, sigma_y)
    epsilon_y = sigma_y*np.random.randn(T)
    df.iloc[0,:] = [x[0], y1, epsilon_y[0]]
    
    for i in range(0, T-1):
        df.iloc[i+1,0] = x[i+1] 
        # Xt ~ N(0,1)
        df.iloc[i+1,1] = alpha_y + beta_y * df.iloc[i,1] + gamma * df.iloc[i+1,0] + epsilon_y[i+1] + theta * epsilon_y[i] # generate Y(t) recursively
        # Yt = α + βYt-1 + γXt + εt + θεt-1
        df.iloc[i+1,2] = epsilon_y[i+1]
    
    return df

In [4]:
df = generate_data(1000, 0.2, 0.5, 0.3, 0.9, 1, 0, seed=42)
print(df)

            x         y epsilon_y
0    0.496714  1.399355  0.924634
1   -0.138264  1.749999   0.05963
2    0.647689  0.676037 -0.646937
3     1.52303  1.110908  0.698223
4   -0.234153  1.707094  0.393485
..        ...       ...       ...
995   -0.2811  2.353144 -0.026521
996  1.797687  1.010134 -0.881875
997  0.640843 -0.059434 -0.163067
998 -0.571179 -0.892734 -0.744903
999  0.572583 -1.420183 -0.675178

[1000 rows x 3 columns]


In [8]:
''' EM Algorithm'''
p = 0.4
M = 1000
n = M
T = 1000
df = df.astype('float64')  # convert data to float64 data type
y = df['y'].values
x = df['x'].values 
y_lag = np.roll(y, 1)
y_lag[0] = 0
y_lag = y_lag.reshape((-1, 1))
#X = np.column_stack((np.ones_like(x), y_lag, x))

τ_2 = 2 / p * (1 - p)
τ = np.sqrt(τ_2)
θ = (1 - 2 * p) / (p * (1 - p))

# Initial values
μ = np.empty(M)  # Empty array of length M
μ[0] = 1  # Set initial value μ_0 = 1
λ = np.empty(M)
λ[0] = 1
m = np.empty(M)
m[0] = 1
#Σ = np.zeros((3, 3))
α = np.empty(M)
α[0] = 1
γ = np.empty(M)
γ[0] = 1

mode_z = np.empty(M)
mode_σ = np.empty(M)
σ = np.empty(M)
β = np.zeros((M, 1))
print(β.shape)
z = np.zeros(M)
l = np.ones((1000, 1))
T = 1000  # Number of rows in your dataset

Σ = np.zeros((T, T))
X = np.zeros((T, 3))
X[:, 0] = np.ones(T)
X[:, 1] = y_lag[:, 0]
X[:, 2] = x

for i in range(M):
    # E-step: estimate mode_z
    mode_z[i] = μ[i] * ((1 + (9 * μ[i] ** 2) / (4 * λ[i] ** 2)) - (3 * μ[i]) / (4 * λ[i]))
    
    # Set mode_z = z_i and input in the parameters of the distribution of σ_i
    z[i] = mode_z[i] 
    α[i+1] = α[i] * (3 * n / 2)
    γ[i+1] = γ[i] + ((y[i+1] - x[i+1] * β[i]) ** 2) / (2 * z[i] * τ_2) + z[i]

    # Estimate the mode mode_σ_{i+1}
    mode_σ[i+1] = γ[i+1] / α[i+1] + 1

    # Set mode_σ_{i+1} = σ_{i+1} and compute U_{i+1}
    σ[i+1] = mode_σ[i+1]

    j = (σ[i+1] * τ_2 * z[i]) ** -1
    
    U = np.zeros((T, T))
    np.fill_diagonal(U, j)

    Σ = np.linalg.inv(X.T @ U @ X)

    r = θ / (σ[i+1] * τ_2)
    r_matr = np.zeros((3, 1))
    r_matr[0, 0] = r

    m[i+1] = Σ @ (X.T @ U @ y.reshape((-1, 1))).flatten() - (X.T @ U @ (r_matr @ l))
    β[i+1] = m[i+1, 0]

    μ[i+1] = np.sqrt(θ ** 2 + 2 * τ_2) / np.abs(y[i+1] - (x[i+1] * β[i + 1]))
    λ[i+1] = (θ ** 2 + 2 * τ_2) / (σ[i + 1] * τ_2)


print(β)

(1000, 1)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1000 is different from 1)